### Предобработка данных

In [1]:
!pip install -q ffmpeg-python

In [2]:
!pip install -q silero-vad

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.

In [3]:
import os
from google.colab import files
# import json
import ffmpeg

In [4]:
VIDEO1_PATH = "Bill_Gates_TED.mp4"
VIDEO2_PATH = "Kat_Cole_TED.mp4"
MIN_SILENCE_LEN = 1000
SILENCE_THRESH = -40


os.makedirs("output_segments/segments1", exist_ok=True)
os.makedirs("output_segments/segments2", exist_ok=True)

In [5]:
def extract_audio(video_path, audio_path):
    ffmpeg.input(video_path).output(
        audio_path,
        format="wav",
        ac=1,         # mono
        ar=16000,     # 16kHz
        vn=None
    ).run(overwrite_output=True)

In [68]:
# def segment_audio_robust(audio_path, output_dir, min_silence_len=500, silence_thresh_offset=14, min_chunk_len=5000, max_chunk_len=30000):
#     audio = AudioSegment.from_file(audio_path, format="wav")
#     silence_thresh = audio.dBFS - silence_thresh_offset

#     print(f"💡 silence_thresh = {silence_thresh:.2f} dBFS")
#     chunks = split_on_silence(audio,
#         min_silence_len=min_silence_len,
#         silence_thresh=silence_thresh,
#         keep_silence=150  # сохраняем немного тишины в начале/конце сегмента
#     )

#     # fallback: если split_on_silence не сработал — режем по таймеру
#     if not chunks or sum(len(c) for c in chunks) < len(audio) * 0.6:
#         print("⚠️ Недостаточно сегментов — делим по времени (fixed chunk size)")
#         chunks = [audio[i:i + max_chunk_len] for i in range(0, len(audio), max_chunk_len)]

#     os.makedirs(output_dir, exist_ok=True)
#     metadata = []
#     cursor = 0

#     return chunks

In [6]:
extract_audio(VIDEO1_PATH, "output_segments/extracted_audio1.wav")
extract_audio(VIDEO2_PATH, "output_segments/extracted_audio2.wav")

In [7]:
import torch
import torchaudio
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps, collect_chunks

model = load_silero_vad()

wav1 = read_audio("output_segments/extracted_audio1.wav", sampling_rate=16000)
speech_timestamps1 = get_speech_timestamps(
  wav1,
  model,
  sampling_rate=16000
)

wav2 = read_audio("output_segments/extracted_audio2.wav", sampling_rate=16000)
speech_timestamps2 = get_speech_timestamps(
  wav2,
  model,
  sampling_rate=16000
)

In [8]:
def save_segments(wav, timestamps, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    for i, t in enumerate(timestamps):
        segment = wav[t["start"]:t["end"]]
        segment = segment.unsqueeze(0)
        torchaudio.save(f"{out_dir}/segment_{i:03d}.wav", segment, 16000)

save_segments(wav1, speech_timestamps1, "output_segments/segments1")
save_segments(wav2, speech_timestamps2, "output_segments/segments2")